In [2]:
import json
import requests
import os
import shutil
import sys
sys.path.append('../get_orthophoto')

from project_metadata import get_all_projects
from start_export import start_export, save_export_job
from status_export import status_export, save_download_url
from download_project import download_project

In [3]:
projects = get_all_projects()
projects[100:120]

['Klepp Time Hå 2023',
 'Sandnes 2023',
 'Lyngdal 2023',
 'Lillesand GSD5 2023',
 'Farsund midlertidig 2023',
 'Farsund 2023',
 'Arendal GSD7 2023',
 'Stavanger 2023',
 'Botn aust 2022',
 'Fjellsteggen 2022',
 'Holtardalen 2022',
 'Heimvegli 2022',
 'Harrkjellstaullia 2022',
 'E39 Kristiansand Vest - Mandal Øst 2022',
 'Kjønstadmarka 2022',
 'Reinsholm 2022',
 'Holåsen 2022',
 'Trøndelag Vest CIR 2022',
 'Trøndelag Vest 2022',
 'Oslo-Østlandet CIR 2022']

In [14]:
def get_project_metadata(project:str)->dict:
    '''
    Get the metadata of the orthophoto project specified.
    Seems to not work as of now (05.03.2024) - not clear if the purpose of this service
    is to get medata back for specific projects, or just to give a list of projects fitting the
    search criteria.

    Arguments:
    - project: The project ID of the orthophoto to get metadata from.

    Returns:
    - A dictionary containing the metadata of the orthophoto project.
    '''
    rest_metatdata_url = "http://tjenester.norgeibilder.no/rest/projectMetadata.ashx" 
    metadata_payload = {
        "Projects": project,
        "ReturnMedata":'True',
        "ReturnGeometry":'True',
        }
    print(metadata_payload)
    metadata_payload_json = json.dumps(metadata_payload)
    metadata_query = {"json": metadata_payload_json}
    meta_data_response = requests.get(rest_metatdata_url, params = metadata_query)

    if meta_data_response.status_code != 200:
        raise Exception(f"Metadata request failed with status code {meta_data_response.status_code}.")
    else:
        metadata_response_dict = meta_data_response.json()

    return metadata_response_dict



In [16]:
system_answer = get_project_metadata('Stavanger 2023')
#print(f'we get a response for these : {system_answer.keys()}')
for value, key in system_answer.items():
    if key != 'ProjectList':
        print(value,":", key)

{'Projects': 'Stavanger 2023', 'ReturnMedata': 'True', 'ReturnGeometry': 'True'}
<class 'dict'>
Success : True
ErrorMessage : None
ProjectList : ['Kyken 2023', 'Sogn CIR 2023', 'Sogn 2023', 'Sunnmøre sør 2023', 'Innlandet NØ 2023', 'Innlandet NØ CIR 2023', 'Meråker Tydal 2023', 'Sunnfjord Mellombels 2023', 'Valdres 2023', 'Øst-Telemark 2023', 'Øst-Telemark CIR 2023', 'Øst-Telemark MOF 2023', 'Innsvatnet 2023', 'Juvfonne 2023', 'Sentinel-2 Skyfri mosaikk 2023', 'Troms CIR 2023', 'Troms 2023', 'Vestfold skråfoto 2023', 'Gausa - Viken Innlandet flom 2023', 'Gausa CIR - Viken Innlandet flom 2023', 'Nedre Glomma øst CIR - Viken flom 2023', 'Valdres CIR - Viken Innlandet flom 2023', 'Ål og Hol - Viken flom 2023', 'Nedre Glomma øst - Viken flom 2023', 'Valdres - Viken flom 2023', 'Ål og Hol CIR - Viken flom 2023', 'Trøndelag NØ CIR 2023', 'Trøndelag NØ 2023', 'Otta-Kvam-Ringebu - Innlandet flom 2023', 'Ottadalen - Viken Innlandet flom 2023', 'Skjåk-Lom-Vågåmo - Innlandet flom 2023', 'Nedre Gl

In [19]:
import json
import requests

def get_project_metadata(project:str)->dict:
    rest_metatdata_url = "http://tjenester.norgeibilder.no/rest/projectMetadata.ashx" 
    metadata_payload = {
        "Projects": project,
        "ReturnMedata":'True',
        "ReturnGeometry":'True',
        }
    print(metadata_payload)
    metadata_payload_json = json.dumps(metadata_payload)
    metadata_query = {"json": metadata_payload_json}
    meta_data_response = requests.get(rest_metatdata_url, params = metadata_query)

    if meta_data_response.status_code != 200:
        raise Exception(f"Metadata request failed with status code {meta_data_response.status_code}.")
    else:
        meta_data_response_dict = meta_data_response.json()

    return meta_data_response_dict, meta_data_response

system_answer_dict, system_anwer= get_project_metadata('Stavanger 2023')
#print(f'we get a response for these : {system_answer.keys()}')
for value, key in system_answer.items():
    if key != 'ProjectList':
        print(value,":", key)

{'Projects': 'Stavanger 2023', 'ReturnMedata': 'True', 'ReturnGeometry': 'True'}
Success : True
ErrorMessage : None
ProjectList : ['Kyken 2023', 'Sogn CIR 2023', 'Sogn 2023', 'Sunnmøre sør 2023', 'Innlandet NØ 2023', 'Innlandet NØ CIR 2023', 'Meråker Tydal 2023', 'Sunnfjord Mellombels 2023', 'Valdres 2023', 'Øst-Telemark 2023', 'Øst-Telemark CIR 2023', 'Øst-Telemark MOF 2023', 'Innsvatnet 2023', 'Juvfonne 2023', 'Sentinel-2 Skyfri mosaikk 2023', 'Troms CIR 2023', 'Troms 2023', 'Vestfold skråfoto 2023', 'Gausa - Viken Innlandet flom 2023', 'Gausa CIR - Viken Innlandet flom 2023', 'Nedre Glomma øst CIR - Viken flom 2023', 'Valdres CIR - Viken Innlandet flom 2023', 'Ål og Hol - Viken flom 2023', 'Nedre Glomma øst - Viken flom 2023', 'Valdres - Viken flom 2023', 'Ål og Hol CIR - Viken flom 2023', 'Trøndelag NØ CIR 2023', 'Trøndelag NØ 2023', 'Otta-Kvam-Ringebu - Innlandet flom 2023', 'Ottadalen - Viken Innlandet flom 2023', 'Skjåk-Lom-Vågåmo - Innlandet flom 2023', 'Nedre Glomma vest-Viken